In [8]:
import numpy as np
import matplotlib.pyplot as plt 

A. Naive

In [1]:
def pack(arr, n=1024):
    newarr = np.zeros(n, dtype=int)
    newarr[:len(arr)] = arr
    return newarr
    
def rotate_sum(arr, n=None, debug=False):
    # Only length of power of 2 is allowed
    assert np.ceil(np.log2(len(arr))) == int(np.log2(len(arr)))\
        , "array length not power of 2. Consider zero-padding"
    
    # if n_valid < n_arr and only single valid output slot is sufficient,
    # I can remove one rotation. -- to be implemented
    if n is None:
        n = len(arr)
    for i in range(int(np.log2(n))):
        arr += np.roll(arr, -2**i)
        if debug:
            print(arr)
    return arr

In [2]:
arr = pack(np.arange(1,11), 2**4)
print(np.sum(arr))
print(arr)
rotate_sum(arr)

NameError: name 'np' is not defined

In [ ]:
ni = 4
no =  4

W0 = np.arange(ni*no).reshape(no,ni)
W1 = [pack(w) for w in W0]
V0 = np.random.randint(1,10,ni)
V1 = pack(V0)

In [3]:
print(W0)
print(W1)

NameError: name 'W0' is not defined

In [4]:
np.matmul(W0,W1)

NameError: name 'np' is not defined

## Output Packing

naive_result의 output을 rotate - sum해서 하나로 합치기. 
-> 근데 에러가 너무 커져서 문제라고? 
곱하기 한번 더 하는 것 뿐인데...  
충분한 정확도를 확보하기 위해 64bit 이상이 필요하고, 그러려면 DCRT가 필요한데 DCRT는 느리고, 알고리즘으로 해결하겠다고 함

### Input Packing

또 다른 방법으론, ni << n일 때, 하나의 ctxt에 데이터를 반복해서 tile하는 방법. 

In [484]:
W3 = pack(W0.ravel())
W3[:ni*no+ni].reshape(no+1,-1)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [ 0,  0,  0,  0]])

In [485]:
V3 = pack(np.tile(V0, ni))
print(V3[:ni+1])

[9 3 7 1 9]


In [486]:
result = rotate_sum(W3*V3, n=ni, debug=False)

result[::ni][:no]

array([ 20, 100, 180, 260])

## Halevi Shoup

In [559]:
def get_diagonal(arr):
    nr, nc = arr.shape
    new_mat = np.zeros((nr, nc), dtype=arr.dtype)
    
    for j in range(nr):
        for i in range(nc):
            new_mat[j, i] = arr[i%nr, (i+j)%nc]
            #new_mat[j, i] = arr[i%nr, (i+j)%nc]
    return new_mat

In [561]:
W4 = get_diagonal(W0)
W4

array([[ 0,  5, 10, 15],
       [ 1,  6, 11, 12],
       [ 2,  7,  8, 13],
       [ 3,  4,  9, 14]])

In [562]:
HS_result = None
for i in range(ni):
    if HS_result is None:
        HS_result = W4[i] * np.roll(V0,-i)
        print(W4[i], np.roll(V0,-i))
    else:
        HS_result += W4[i] * np.roll(V0,-i)
        print(W4[i], np.roll(V0,-i))

[ 0  5 10 15] [3 9 4 6]
[ 1  6 11 12] [9 4 6 3]
[ 2  7  8 13] [4 6 3 9]
[ 3  4  9 14] [6 3 9 4]


In [563]:
print(HS_result)
print(np.matmul(W0, V0))

[ 35 123 211 299]
[ 35 123 211 299]


### Diagonal, rectangular

no = 4이므로, 벡터끼리 곱하기가 ni번 반복되어 no=4인 벡터로 합쳐져야함. 

근데 input vector가 이미 8짜리인데? 

WW가 (4,8)이면 (8,4) 바꿔야함.



[[a,b,c],
 [d,e,f]]
x
[1,2,3]

-> 
[[a,e],
 [b,f],
 [c,d]] 
x
[[1,2],
 [2,3],
 [3,1]]
  
로 바꿔야함.
= [(1a+2b+3c), (2e+3f+1d)]

In [10]:
def get_diagonal(arr):
    nr, nc = arr.shape
    new_mat = np.zeros((nc, nr), dtype=arr.dtype)
    
    for j in range(nr):
        for i in range(nc):
            new_mat[i, j%nr] = arr[i%nr, (i+j)%nc]
            #new_mat[j, i] = arr[i%nr, (i+j)%nc]
    return new_mat

In [11]:
W0

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [12]:
V0

array([7, 3, 2, 2])

In [13]:
W5 = get_diagonal(W0)
W5

array([[ 0,  1,  2,  3],
       [ 5,  6,  7,  4],
       [10, 11,  8,  9],
       [15, 12, 13, 14]])

In [14]:
ni = 3
no = 2

W0 = np.arange(ni*no).reshape(no,ni)
W1 = [pack(w) for w in W0]
V0 = np.random.randint(1,10,ni)
V1 = pack(V0)

In [15]:
W0

array([[0, 1, 2],
       [3, 4, 5]])

In [16]:
W5 = get_diagonal(W0)

W5

array([[0, 1],
       [4, 5],
       [2, 0]])

In [17]:
def diagonal_matmult(WW, VV):
    HS_result = None
    nr, nc = WW.shape
    for i in range(nc):
        if HS_result is None:
            HS_result = WW[i] * VV
        else:
            HS_result += WW[i] * np.roll(VV,-i)
    return HS_result

In [18]:
hs = diagonal_matmult(W5, V0)

ValueError: operands could not be broadcast together with shapes (2,) (3,) 

In [573]:
hs

array([ 39, 247, 418, 596,  85, 197, 346, 488])

In [574]:
np.matmul(W0, V0)

array([ 124,  444,  764, 1084])

### Hybrid method

In [5]:
def get_diagonal(arr):
    nr, nc = arr.shape
    new_mat = np.zeros((nr, nc), dtype=arr.dtype)
    
    for j in range(nr):
        for i in range(nc):
            new_mat[j, i] = arr[(i+j)%nr,i]
    return new_mat

In [20]:
def get_diagonal2(arr, ndiv=2):
    """Hybrid
    """
    nrow, ncol = arr.shape
    nnrow, nncol = int(nrow / ndiv), int(ncol * ndiv)
    assert nrow/ndiv == nrow // ndiv, f"can't divide the matrix by {ndiv}"
    new_mat = arr.transpose((1,0,2)).reshape(nnrow,nncol)
    return new_mat

In [21]:
V0

array([7, 6, 8])

In [22]:
WW = get_diagonal2(W0,2)
for i in range(ni):
    WW * np.roll(V0, -i)

ValueError: axes don't match array

In [9]:
d = 4 

mat1 = np.arange(d*d).reshape((d,d))
mat2 = np.arange(d*d).reshape((d,d))

ans = np.matmul(mat1, mat2)


In [10]:
from hemul.matrix import *

MM = MatrixMultiplicator(d)

In [11]:
emat1 = encode_matrix_to_vector(mat1)
emat2 = encode_matrix_to_vector(mat2)
result = decode_vector_to_matrix(MM.matmul(emat1, emat2))

In [12]:
result

array([[ 56.,  62.,  68.,  74.],
       [152., 174., 196., 218.],
       [248., 286., 324., 362.],
       [344., 398., 452., 506.]])

In [20]:
ans

array([[ 56,  62,  68,  74],
       [152, 174, 196, 218],
       [248, 286, 324, 362],
       [344, 398, 452, 506]])

In [13]:
np.all(result == ans)

True